In [ ]:
import os
from utils import *
import utils
from agents import *
import time
import torch
import torch.nn as nn
from copy import deepcopy
import argparse

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--seeds',          type=int,       default=[2023, 2024, 2025])
parser.add_argument('--dataset',        type=str,       default='fmnist')
parser.add_argument('--batch_size',     type=int,       default=512)
parser.add_argument('--model_name',     type=str,       default='alexnet')
parser.add_argument('--retrain',        type=bool,      default=False)
parser.add_argument('--unlearn_class',  type=list,      default=3)
args = parser.parse_args("")
args.time_str = time.strftime("%m-%d-%H-%M", time.localtime())
if args.dataset.lower() == 'fmnist':
    args.n_channels = 1
else:
    args.n_channels = 3

if args.dataset.lower() == 'cifar100':
    args.num_classes = 100
else:
    args.num_classes = 10
criterion = nn.CrossEntropyLoss()

In [3]:
dict = {
    1: {'unlean_class': [6], 'arxiv_name': '12-27-21-31'},
    2: {'unlean_class': [0, 6], 'arxiv_name': '12-28-20-44'}, 
    3: {'unlean_class': [0, 4, 6], 'arxiv_name': '12-28-22-00'},
    4: {'unlean_class': [0, 1, 4, 6], 'arxiv_name': '12-28-22-51'},
    5: {'unlean_class': [0, 1, 4, 6, 9], 'arxiv_name': '12-28-23-25'},
}

num_unlearn = 1
args.unlean_class = dict[num_unlearn]['unlean_class']
arxiv_name = dict[num_unlearn]['arxiv_name']

arxiv_name_o = '12-27-20-22'

# Unlearn 1 classes

In [4]:
num_unlearn = 1
arxiv_name = dict[num_unlearn]['arxiv_name']
args.unlearn_class = dict[num_unlearn]['unlean_class']
train_loader, val_loader, test_loader = get_dataloader(args)
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


0.8490, 0.9780, 0.9090, 0.9310, 0.8380, 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8260, Acc_r: 0.9326
0.8460, 0.9820, 0.9160, 0.9370, 0.8760, 0.9640, [0.8120], 0.9880, 0.9850, 0.9610, Acc_f: 0.8120, Acc_r: 0.9394
0.8580, 0.9760, 0.8940, 0.9400, 0.8640, 0.9570, [0.7910], 0.9830, 0.9850, 0.9610, Acc_f: 0.7910, Acc_r: 0.9353
------------ Retrained model ------------
0.9080, 0.9820, 0.9320, 0.9510, 0.9010, 0.9560, [0.0000], 0.9830, 0.9830, 0.9690, Acc_f: 0.0000, Acc_r: 0.9517
0.9230, 0.9800, 0.9420, 0.9410, 0.8960, 0.9360, [0.0000], 0.9770, 0.9860, 0.9650, Acc_f: 0.0000, Acc_r: 0.9496
0.9330, 0.9810, 0.9450, 0.9430, 0.8950, 0.9570, [0.0000], 0.9730, 0.9930, 0.9700, Acc_f: 0.0000, Acc_r: 0.9544
Original model Acc_f: 80.97 \pm 1.44
Original model Acc_r: 93.58 \pm 0.28
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 95.19 \pm 0.20


In [6]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Proj_mat_lst =[]
train_targets_list = np.array(train_loader.dataset.targets)[train_loader.sampler.indices]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 125, 125, 250, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)

------------ Trail 0 ------------
Threshold:  0.97
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 3/16
Layer 2 : 101/576
Layer 3 : 284/512
Layer 4 : 91/1024
Layer 5 : 86/2048
Layer 6 : 6/2048
----------------------------------------
Threshold:  0.973
Skip Updating GPM for layer: 2
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 101/576
Layer 3 : 343/512
Layer 4 : 162/1024
Layer 5 : 129/2048
Layer 6 : 7/2048
----------------------------------------
Threshold:  0.976
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/16
Layer 2 : 139/576
Layer 3 : 394/512
Layer 4 : 252/1024
Layer 5 : 211/2048
Layer 6 : 11/2048
----------------------------------------
Threshold:  0.979
Skip Updating GPM for layer: 1
----------------------------------------
Gradient Constraint

In [7]:
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(10):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = torch.from_numpy(np.random.choice(remain_class, size=x.shape[0])).cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.8490, 0.9780, 0.9090, 0.9310, 0.8380, 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8260, Acc_r: 0.9326
[train] epoch 0, batch 10, loss 5.916708946228027
0.8740, 0.9800, 0.9400, 0.9530, 0.8950, 0.9570, [0.6140], 0.9850, 0.9890, 0.9590, Acc_f: 0.6140, Acc_r: 0.9480
[train] epoch 1, batch 10, loss 3.784639596939087
0.8830, 0.9830, 0.9400, 0.9590, 0.8970, 0.9540, [0.2740], 0.9850, 0.9900, 0.9590, Acc_f: 0.2740, Acc_r: 0.9500
[train] epoch 2, batch 10, loss 3.376570224761963
0.8860, 0.9830, 0.9430, 0.9640, 0.8900, 0.9530, [0.1690], 0.9850, 0.9880, 0.9590, Acc_f: 0.1690, Acc_r: 0.9501
[train] epoch 3, batch 10, loss 3.2416114807128906
0.8920, 0.9830, 0.9420, 0.9620, 0.8840, 0.9530, [0.1130], 0.9850, 0.9870, 0.9590, Acc_f: 0.1130, Acc_r: 0.9497
[train] epoch 4, batch 10, loss 3.068338632583618
0.8940, 0.9830, 0.9410, 0.9580, 0.8870, 0.9530, [0.0830], 0.9860, 0.9870, 0.9620, Acc_f: 0.0830, Acc_r: 0.9501
[train] epoch 5, batch 10, loss 2.955772876739502
0.8960, 0.9850, 0.9370, 0.9560, 0

In [9]:
Acc_r = 100*np.array([0.9482, 0.9289, 0.9218])
Acc_f = 100*np.array([0.0160, 0.0090, 0.0220])

print(f'Random label + Subspace Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Random label + Subspace Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Random label + Subspace Acc_f: 1.57 \pm 0.53
Random label + Subspace Acc_r: 93.30 \pm 1.12


In [8]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.8490, 0.9780, 0.9090, 0.9310, 0.8380, 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8260, Acc_r: 0.9326
[train] epoch 0, batch 10, loss 2.2840707302093506
0.8740, 0.9780, 0.9460, 0.9370, 0.9090, 0.9560, [0.4840], 0.9850, 0.9890, 0.9590, Acc_f: 0.4840, Acc_r: 0.9481
[train] epoch 1, batch 10, loss 1.2306489944458008
0.8870, 0.9790, 0.9550, 0.9400, 0.9070, 0.9520, [0.2470], 0.9850, 0.9890, 0.9590, Acc_f: 0.2470, Acc_r: 0.9503
[train] epoch 2, batch 10, loss 0.9398791193962097
0.8940, 0.9790, 0.9560, 0.9450, 0.8980, 0.9500, [0.1740], 0.9850, 0.9890, 0.9590, Acc_f: 0.1740, Acc_r: 0.9506
[train] epoch 3, batch 10, loss 0.7715941071510315
0.9060, 0.9790, 0.9560, 0.9450, 0.8930, 0.9500, [0.1340], 0.9860, 0.9890, 0.9590, Acc_f: 0.1340, Acc_r: 0.9514
[train] epoch 4, batch 10, loss 0.5707158446311951
0.9130, 0.9800, 0.9550, 0.9450, 0.8900, 0.9490, [0.0970], 0.9860, 0.9890, 0.9590, Acc_f: 0.0970, Acc_r: 0.9518
[train] epoch 5, batch 10, loss 0.5738160014152527
0.9160, 0.9800, 0.9530, 0.94

In [3]:

Acc_r = 100*np.array([0.9531, 0.9551, 0.9487])
Acc_f = 100*np.array([0.0090, 0.0060, 0.0050])

print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

UNSC Acc_f: 0.67 \pm 0.17
UNSC Acc_r: 95.23 \pm 0.27


# Boundary Unlearning

In [12]:
from agents.adv import FGSM

def find_adjacent_cls(adv_agent, x, y):
    x_adv = adv_agent.perturb(x, y)
    adv_logits = model(x_adv)
    adv_pred = torch.argmax(adv_logits.data, 1)
    return adv_pred, x_adv

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    adv_agent = FGSM(deepcopy(model), bound=0.5, norm=False, random_start=True, device='cuda')
    sgd_mr_model = deepcopy(model)
    print('==='*20)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0005)

    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()
            
    model.eval()
    for ep in range(20):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            adv_pred, x_adv = find_adjacent_cls(adv_agent, x, y)
            adv_y = torch.argmax(model(x_adv), dim=1).detach().cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, adv_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.8490, 0.9780, 0.9090, 0.9310, 0.8380, 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8260, Acc_r: 0.9326
[train] epoch 0, batch 10, loss 1.9312938451766968
0.1170, 0.9600, 0.8500, 0.7780, 0.2310, 0.8550, [0.0930], 0.9180, 0.9940, 0.9750, Acc_f: 0.0930, Acc_r: 0.7420
[train] epoch 1, batch 10, loss 1.0576661825180054
0.0660, 0.9600, 0.9080, 0.8080, 0.2010, 0.8280, [0.0690], 0.9180, 0.9870, 0.9750, Acc_f: 0.0690, Acc_r: 0.7390
[train] epoch 2, batch 10, loss 1.120916724205017
0.0570, 0.9590, 0.9000, 0.8630, 0.1910, 0.8150, [0.0640], 0.9280, 0.9780, 0.9740, Acc_f: 0.0640, Acc_r: 0.7406
[train] epoch 3, batch 10, loss 0.9497131109237671
0.0500, 0.9580, 0.8930, 0.8880, 0.1810, 0.8030, [0.0510], 0.9340, 0.9740, 0.9690, Acc_f: 0.0510, Acc_r: 0.7389
[train] epoch 4, batch 10, loss 0.9455651640892029
0.0380, 0.9570, 0.8920, 0.8950, 0.1650, 0.7810, [0.0440], 0.9340, 0.9660, 0.9680, Acc_f: 0.0440, Acc_r: 0.7329
[train] epoch 5, batch 10, loss 0.9393824934959412
0.0340, 0.9560, 0.8880, 0.897

In [18]:
Acc_r = 100*np.array([0.7329, 0.7262, 0.7223])
Acc_f = 100*np.array([0.0440, 0.0130, 0.0320])

print(f'BU Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'BU Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

BU Acc_f: 2.97 \pm 1.28
BU Acc_r: 72.71 \pm 0.44


# RL

In [20]:
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(10):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = torch.from_numpy(np.random.choice(remain_class, size=x.shape[0])).cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.8490, 0.9780, 0.9090, 0.9310, 0.8380, 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8260, Acc_r: 0.9326
[train] epoch 0, batch 10, loss 4.493492603302002
0.6980, 0.9780, 0.7870, 0.9150, 0.7260, 0.9500, [0.1230], 0.9810, 0.9910, 0.9620, Acc_f: 0.1230, Acc_r: 0.8876
[train] epoch 1, batch 10, loss 2.921926259994507
0.7990, 0.9850, 0.8450, 0.9450, 0.7630, 0.9530, [0.0420], 0.9730, 0.9860, 0.9660, Acc_f: 0.0420, Acc_r: 0.9128
[train] epoch 2, batch 10, loss 2.778705596923828
0.8170, 0.9870, 0.8540, 0.9470, 0.7750, 0.9550, [0.0240], 0.9640, 0.9850, 0.9730, Acc_f: 0.0240, Acc_r: 0.9174
[train] epoch 3, batch 10, loss 2.756378650665283
0.8250, 0.9880, 0.8450, 0.9460, 0.7610, 0.9530, [0.0140], 0.9570, 0.9840, 0.9760, Acc_f: 0.0140, Acc_r: 0.9150
[train] epoch 4, batch 10, loss 2.516944646835327
0.8180, 0.9890, 0.8400, 0.9400, 0.7630, 0.9490, [0.0080], 0.9530, 0.9840, 0.9770, Acc_f: 0.0080, Acc_r: 0.9126
[train] epoch 5, batch 10, loss 2.4788084030151367
0.8290, 0.9890, 0.8480, 0.9340, 0

In [22]:
Acc_r = 100*np.array([0.9174, 0.9153, 0.9124])
Acc_f = 100*np.array([0.0240, 0.0230, 0.0540])

print(f'RL Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'RL Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

RL Acc_f: 3.37 \pm 1.44
RL Acc_r: 91.50 \pm 0.20


# Gradient Ascent

In [31]:
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')

    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.00005)
    
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()
            
    for ep in range(30):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = y.cuda()
            pred_y = sgd_mr_model(x)
            loss = -criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        print('>> unlearned model testing acc by class SGD <<')
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

------------ Trail 0 ------------
0.8490, 0.9780, 0.9090, 0.9310, 0.8380, 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8260, Acc_r: 0.9326
[train] epoch 0, batch 10, loss -0.11080976575613022
>> unlearned model testing acc by class SGD <<
0.8570, 0.9780, 0.9140, 0.9320, 0.8450, 0.9580, [0.8130], 0.9850, 0.9880, 0.9570, Acc_f: 0.8130, Acc_r: 0.9349
[train] epoch 1, batch 10, loss -0.056481387466192245
>> unlearned model testing acc by class SGD <<
0.8600, 0.9780, 0.9160, 0.9300, 0.8480, 0.9580, [0.8050], 0.9850, 0.9880, 0.9570, Acc_f: 0.8050, Acc_r: 0.9356
[train] epoch 2, batch 10, loss -0.053473953157663345
>> unlearned model testing acc by class SGD <<
0.8640, 0.9780, 0.9190, 0.9280, 0.8480, 0.9580, [0.7970], 0.9850, 0.9880, 0.9570, Acc_f: 0.7970, Acc_r: 0.9361
[train] epoch 3, batch 10, loss -0.05863038823008537
>> unlearned model testing acc by class SGD <<
0.8670, 0.9780, 0.9190, 0.9280, 0.8490, 0.9580, [0.7880], 0.9850, 0.9880, 0.9570, Acc_f: 0.7880, Acc_r: 0.9366
[train] e

In [32]:
Acc_r = 100*np.array([0.7548, 0.8404, 0.8330])
Acc_f = 100*np.array([0.0990, 0.0650, 0.0280])

print(f'GA Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'GA Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

GA Acc_f: 6.40 \pm 2.90
GA Acc_r: 80.94 \pm 3.87


# Fisher

In [24]:
import copy
import torch.nn.functional as F

def hessian(model, train_loader):
    model.eval()
    loss_fn = torch.nn.CrossEntropyLoss(reduction="mean")
    device = torch.device("cuda")

    for p in model.parameters():
        p.grad2_acc = 0
    
    for data, orig_target in tqdm(train_loader):
        data, orig_target = data.to(device), orig_target.to(device)
        output = model(data)
        prob = F.softmax(output, dim=-1).data

        for y in range(output.shape[1]):
            target = torch.empty_like(orig_target).fill_(y)
            loss = loss_fn(output, target)
            model.zero_grad()
            loss.backward(retain_graph=True)
            for p in model.parameters():
                if p.requires_grad:
                    p.grad2_acc += torch.mean(prob[:, y]) * p.grad.data.pow(2) 

    for p in model.parameters():
        p.grad2_acc /= len(train_loader)
    
def get_mean_var(args, p, alpha=1e-7):
    var = copy.deepcopy(1./(p.grad2_acc+1e-8))
    var = var.clamp(max=1e3) 
    if p.size(0) == args.num_classes:
        var = var.clamp(max=1e2)
    var = alpha * var 
    
    if p.ndim > 1:
        var = var.mean(dim=1, keepdim=True).expand_as(p).clone()
    mu = copy.deepcopy(p.data0.clone())

    if p.size(0) == args.num_classes:
        mu[args.unlearn_class] = 0
        var[args.unlearn_class] = 0.0001
        var *= 10
    elif p.ndim == 1:
        var *= 10 
    return mu, var

def fisher_new(model, train_loader):
    for p in model.parameters():
        p.data0 = copy.deepcopy(p.data.clone())
    hessian(model, train_loader)
    for i, p in enumerate(model.parameters()):
        mu, var = get_mean_var(args, p)
        p.data = mu + var.sqrt() * torch.empty_like(p.data).normal_()
    return model

In [26]:
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    test_by_class(model, test_loader, i=args.unlearn_class)
    fisher_model = copy.deepcopy(model)
    fisher_new(fisher_model, remain_train_loader)
    test_by_class(fisher_model, test_loader, i=args.unlearn_class)

0.8490, 0.9780, 0.9090, 0.9310, 0.8380, 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8260, Acc_r: 0.9326


100%|██████████| 95/95 [00:05<00:00, 16.56it/s]


0.9390, 0.9820, 0.9510, 0.9380, 0.8440, 0.9570, [0.0310], 0.9820, 0.9920, 0.9560, Acc_f: 0.0310, Acc_r: 0.9490
0.8460, 0.9820, 0.9160, 0.9370, 0.8760, 0.9640, [0.8120], 0.9880, 0.9850, 0.9610, Acc_f: 0.8120, Acc_r: 0.9394


100%|██████████| 95/95 [00:05<00:00, 17.00it/s]


0.9240, 0.9820, 0.9620, 0.9340, 0.8560, 0.9690, [0.0140], 0.9780, 0.9880, 0.9680, Acc_f: 0.0140, Acc_r: 0.9512
0.8580, 0.9760, 0.8940, 0.9400, 0.8640, 0.9570, [0.7910], 0.9830, 0.9850, 0.9610, Acc_f: 0.7910, Acc_r: 0.9353


100%|██████████| 95/95 [00:05<00:00, 16.68it/s]


0.9280, 0.9810, 0.9400, 0.9550, 0.8930, 0.9600, [0.0000], 0.9780, 0.9810, 0.9630, Acc_f: 0.0000, Acc_r: 0.9532


In [29]:
Acc_r = 100*np.array([0.9490, 0.9512, 0.9532])
Acc_f = 100*np.array([0.0310, 0.0140, 0.0000])

print(f'Fisher Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'Fisher Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

Fisher Acc_f: 1.50 \pm 1.27
Fisher Acc_r: 95.11 \pm 0.17


# SalUn

In [34]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)

remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

In [36]:
# create saliency map
def save_gradient_ratio(unlearn_train_loader, model, criterion, args, seed):
    optimizer = torch.optim.SGD(
        model.parameters(),
        args.unlearn_lr,
        momentum=args.momentum,
        weight_decay=args.weight_decay,
    )
    gradients = {}
    model.eval()
    for name, param in model.named_parameters():
        gradients[name] = 0

    for i, (image, target) in enumerate(unlearn_train_loader):
        image = image.cuda()
        target = target.cuda()

        # compute output
        output_clean = model(image)
        loss = - criterion(output_clean, target)

        optimizer.zero_grad()
        loss.backward()

        with torch.no_grad():
            for name, param in model.named_parameters():
                if param.grad is not None:
                    gradients[name] += param.grad.data

    with torch.no_grad():
        for name in gradients:
            gradients[name] = torch.abs_(gradients[name])

    threshold_list = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

    for i in threshold_list:
        print(i)
        sorted_dict_positions = {}
        hard_dict = {}

        # Concatenate all tensors into a single tensor
        all_elements = - torch.cat([tensor.flatten() for tensor in gradients.values()])

        # Calculate the threshold index for the top 10% elements
        threshold_index = int(len(all_elements) * i)

        # Calculate positions of all elements
        positions = torch.argsort(all_elements)
        ranks = torch.argsort(positions)

        start_index = 0
        for key, tensor in gradients.items():
            num_elements = tensor.numel()
            # tensor_positions = positions[start_index: start_index + num_elements]
            tensor_ranks = ranks[start_index : start_index + num_elements]

            sorted_positions = tensor_ranks.reshape(tensor.shape)
            sorted_dict_positions[key] = sorted_positions

            # Set the corresponding elements to 1
            threshold_tensor = torch.zeros_like(tensor_ranks)
            threshold_tensor[tensor_ranks < threshold_index] = 1
            threshold_tensor = threshold_tensor.reshape(tensor.shape)
            hard_dict[key] = threshold_tensor
            start_index += num_elements

        torch.save(hard_dict, f'./save/{args.dataset}/{args.model_name}/mask_threshold_{seed}_{i}.pt')


args.unlearn_lr=0.01
args.momentum=0.9
args.weight_decay=5e-4
criterion = nn.CrossEntropyLoss()

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    save_gradient_ratio(unlearn_train_loader, model, criterion, args, args.seeds[i])

0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


In [40]:
from agents.svc_mia import SVC_MIA
indice = remain_train_loader.sampler.indices
neg_size = len(test_loader.sampler) + len(val_loader.sampler.indices)
balanced_indice = np.random.choice(indice, size=neg_size, replace=False)
balanced_sampler = torch.utils.data.SubsetRandomSampler(balanced_indice)
balanced_train_loader = torch.utils.data.DataLoader(remain_train_loader.dataset,
                                                    batch_size=args.batch_size,
                                                    sampler=balanced_sampler)

threshold = 0.8

for i in range(3):
    print("======="*50)
    mask = torch.load(f'./save/{args.dataset}/{args.model_name}/mask_threshold_{args.seeds[i]}_{threshold}.pt')
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.0001)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    for ep in range(20):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = torch.from_numpy(np.random.choice(remain_class, size=x.shape[0])).cuda()
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            for name, param in sgd_mr_model.named_parameters():
                if param.grad is not None:
                    param.grad *= mask[name]
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

0.8490, 0.9780, 0.9090, 0.9310, 0.8380, 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8260, Acc_r: 0.9326
[train] epoch 0, batch 10, loss 7.825351715087891
0.8770, 0.9790, 0.9350, 0.9340, 0.8280, 0.9550, [0.5690], 0.9850, 0.9910, 0.9600, Acc_f: 0.5690, Acc_r: 0.9382
[train] epoch 1, batch 10, loss 4.5068488121032715
0.8540, 0.9760, 0.8840, 0.9220, 0.7520, 0.9450, [0.3040], 0.9850, 0.9920, 0.9620, Acc_f: 0.3040, Acc_r: 0.9191
[train] epoch 2, batch 10, loss 3.8723034858703613
0.8230, 0.9770, 0.8670, 0.9190, 0.7440, 0.9440, [0.2000], 0.9850, 0.9920, 0.9630, Acc_f: 0.2000, Acc_r: 0.9127
[train] epoch 3, batch 10, loss 3.56961989402771
0.8110, 0.9790, 0.8650, 0.9230, 0.7530, 0.9420, [0.1510], 0.9850, 0.9880, 0.9620, Acc_f: 0.1510, Acc_r: 0.9120
[train] epoch 4, batch 10, loss 3.3138904571533203
0.8090, 0.9810, 0.8650, 0.9250, 0.7560, 0.9430, [0.1110], 0.9850, 0.9860, 0.9620, Acc_f: 0.1110, Acc_r: 0.9124
[train] epoch 5, batch 10, loss 3.21722412109375
0.8040, 0.9830, 0.8630, 0.9280, 0

In [43]:
Acc_r = 100*np.array([0.9156, 0.9081, 0.9097])
Acc_f = 100*np.array([0.0110, 0.0050, 0.0380])

print(f'SalUn Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'SalUn Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

SalUn Acc_f: 1.80 \pm 1.44
SalUn Acc_r: 91.11 \pm 0.32


# Unlearn 2 classes

In [41]:
num_unlearn = 2
arxiv_name = dict[num_unlearn]['arxiv_name']
args.unlearn_class = dict[num_unlearn]['unlean_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


[0.8490], 0.9780, 0.9090, 0.9310, 0.8380, 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8375, Acc_r: 0.9430
[0.8460], 0.9820, 0.9160, 0.9370, 0.8760, 0.9640, [0.8120], 0.9880, 0.9850, 0.9610, Acc_f: 0.8290, Acc_r: 0.9511
[0.8580], 0.9760, 0.8940, 0.9400, 0.8640, 0.9570, [0.7910], 0.9830, 0.9850, 0.9610, Acc_f: 0.8245, Acc_r: 0.9450
------------ Retrained model ------------
[0.0000], 0.9820, 0.9530, 0.9600, 0.9170, 0.9430, [0.0000], 0.9860, 0.9880, 0.9620, Acc_f: 0.0000, Acc_r: 0.9614
[0.0000], 0.9830, 0.9600, 0.9550, 0.9130, 0.9420, [0.0000], 0.9840, 0.9890, 0.9640, Acc_f: 0.0000, Acc_r: 0.9612
[0.0000], 0.9800, 0.9520, 0.9540, 0.9160, 0.9570, [0.0000], 0.9820, 0.9900, 0.9660, Acc_f: 0.0000, Acc_r: 0.9621
Original model Acc_f: 83.03 \pm 0.54
Original model Acc_r: 94.64 \pm 0.35
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 96.16 \pm 0.04


In [42]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Proj_mat_lst =[]
train_targets_list = np.array(train_loader.dataset.targets)[train_loader.sampler.indices]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100,  125, 125, 250,  256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)

------------ Trail 0 ------------
Threshold:  0.973


----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 2/16
Layer 2 : 52/576
Layer 3 : 224/512
Layer 4 : 71/1024
Layer 5 : 69/2048
Layer 6 : 3/2048
----------------------------------------
Threshold:  0.976
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/16
Layer 2 : 130/576
Layer 3 : 366/512
Layer 4 : 165/1024
Layer 5 : 211/2048
Layer 6 : 8/2048
----------------------------------------
Threshold:  0.979
Skip Updating GPM for layer: 1
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/16
Layer 2 : 148/576
Layer 3 : 421/512
Layer 4 : 260/1024
Layer 5 : 371/2048
Layer 6 : 13/2048
----------------------------------------
Threshold:  0.982
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 6/16
Layer 2 : 175/576

In [44]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.001)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

[0.8490], 0.9780, 0.9090, 0.9310, 0.8380, 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8375, Acc_r: 0.9430
[train] epoch 0, batch 21, loss 2.983308792114258
[0.2620], 0.9790, 0.9590, 0.9310, 0.8840, 0.9510, [0.6050], 0.9840, 0.9880, 0.9630, Acc_f: 0.4335, Acc_r: 0.9549
[train] epoch 1, batch 21, loss 1.239094614982605
[0.0970], 0.9790, 0.9700, 0.9350, 0.8890, 0.9440, [0.3320], 0.9850, 0.9890, 0.9630, Acc_f: 0.2145, Acc_r: 0.9567
[train] epoch 2, batch 21, loss 0.6775612235069275
[0.0550], 0.9800, 0.9670, 0.9400, 0.8870, 0.9370, [0.1860], 0.9830, 0.9900, 0.9640, Acc_f: 0.1205, Acc_r: 0.9560
[train] epoch 3, batch 21, loss 0.5368953347206116
[0.0330], 0.9790, 0.9630, 0.9440, 0.8800, 0.9310, [0.1210], 0.9830, 0.9900, 0.9660, Acc_f: 0.0770, Acc_r: 0.9545
[train] epoch 4, batch 21, loss 0.5600632429122925
[0.0230], 0.9790, 0.9620, 0.9460, 0.8770, 0.9280, [0.0650], 0.9820, 0.9900, 0.9660, Acc_f: 0.0440, Acc_r: 0.9537
[train] epoch 5, batch 21, loss 0.289654403924942
[0.0180], 0.9780, 0

In [7]:
Acc_r = 100*np.array([0.9536, 0.9585, 0.9492])
Acc_f = 100*np.array([0.0080, 0.0045, 0.0115])

print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

UNSC Acc_f: 0.80 \pm 0.29
UNSC Acc_r: 95.38 \pm 0.38


In [3]:
a = (100*np.array([0.9536, 0.9585, 0.9492])-100*np.array([0.9430, 0.9511, 0.9450]))
a.mean(), a.std()

(0.7400000000000043, 0.2612789058968726)

# Unlearn 3 classes

In [47]:
num_unlearn = 3
arxiv_name = dict[num_unlearn]['arxiv_name']
args.unlearn_class = dict[num_unlearn]['unlean_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


[0.8490], 0.9780, 0.9090, 0.9310, [0.8380], 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8377, Acc_r: 0.9580
[0.8460], 0.9820, 0.9160, 0.9370, [0.8760], 0.9640, [0.8120], 0.9880, 0.9850, 0.9610, Acc_f: 0.8447, Acc_r: 0.9619
[0.8580], 0.9760, 0.8940, 0.9400, [0.8640], 0.9570, [0.7910], 0.9830, 0.9850, 0.9610, Acc_f: 0.8377, Acc_r: 0.9566
------------ Retrained model ------------
[0.0000], 0.9810, 0.9850, 0.9630, [0.0000], 0.9600, [0.0000], 0.9770, 0.9910, 0.9670, Acc_f: 0.0000, Acc_r: 0.9749
[0.0000], 0.9820, 0.9820, 0.9610, [0.0000], 0.9520, [0.0000], 0.9870, 0.9900, 0.9580, Acc_f: 0.0000, Acc_r: 0.9731
[0.0000], 0.9830, 0.9850, 0.9670, [0.0000], 0.9470, [0.0000], 0.9860, 0.9920, 0.9630, Acc_f: 0.0000, Acc_r: 0.9747
Original model Acc_f: 84.00 \pm 0.33
Original model Acc_r: 95.88 \pm 0.22
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 97.42 \pm 0.08


In [48]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Proj_mat_lst =[]
train_targets_list = np.array(train_loader.dataset.targets)[train_loader.sampler.indices]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[100, 100,  125, 125, 250,  256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)

------------ Trail 0 ------------
Threshold:  0.973
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 2/16
Layer 2 : 53/576
Layer 3 : 222/512
Layer 4 : 73/1024
Layer 5 : 70/2048
Layer 6 : 3/2048
----------------------------------------
Threshold:  0.976
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/16
Layer 2 : 136/576
Layer 3 : 367/512
Layer 4 : 166/1024
Layer 5 : 215/2048
Layer 6 : 9/2048
----------------------------------------
Threshold:  0.979
Skip Updating GPM for layer: 1
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/16
Layer 2 : 149/576
Layer 3 : 418/512
Layer 4 : 261/1024
Layer 5 : 378/2048
Layer 6 : 13/2048
----------------------------------------
Threshold:  0.985
----------------------------------------
Gradient Constraints Summary
---------------------

In [49]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.001)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

[0.8490], 0.9780, 0.9090, 0.9310, [0.8380], 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8377, Acc_r: 0.9580
[train] epoch 0, batch 31, loss 3.2238874435424805
[0.2820], 0.9790, 0.9750, 0.9440, [0.6410], 0.9490, [0.6190], 0.9820, 0.9890, 0.9660, Acc_f: 0.5140, Acc_r: 0.9691
[train] epoch 1, batch 31, loss 1.3306982517242432
[0.1020], 0.9780, 0.9860, 0.9550, [0.3860], 0.9370, [0.3550], 0.9800, 0.9890, 0.9720, Acc_f: 0.2810, Acc_r: 0.9710
[train] epoch 2, batch 31, loss 0.9686804413795471
[0.0580], 0.9780, 0.9850, 0.9630, [0.2160], 0.9300, [0.1870], 0.9780, 0.9910, 0.9730, Acc_f: 0.1537, Acc_r: 0.9711
[train] epoch 3, batch 31, loss 0.6143273711204529
[0.0350], 0.9760, 0.9800, 0.9710, [0.1210], 0.9260, [0.0920], 0.9770, 0.9910, 0.9730, Acc_f: 0.0827, Acc_r: 0.9706
[train] epoch 4, batch 31, loss 0.602303683757782
[0.0240], 0.9750, 0.9790, 0.9720, [0.0810], 0.9240, [0.0530], 0.9760, 0.9910, 0.9730, Acc_f: 0.0527, Acc_r: 0.9700
[train] epoch 5, batch 31, loss 0.5115252733230591
[0.01

In [9]:
Acc_r = 100*np.array([0.9701, 0.9699, 0.9639])
Acc_f = 100*np.array([0.0057, 0.0093, 0.0197])

print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

UNSC Acc_f: 1.16 \pm 0.59
UNSC Acc_r: 96.80 \pm 0.29


# Unlearn 4 classes

In [52]:
num_unlearn = 4
arxiv_name = dict[num_unlearn]['arxiv_name']
args.unlearn_class = dict[num_unlearn]['unlean_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


[0.8490], [0.9780], 0.9090, 0.9310, [0.8380], 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8728, Acc_r: 0.9547
[0.8460], [0.9820], 0.9160, 0.9370, [0.8760], 0.9640, [0.8120], 0.9880, 0.9850, 0.9610, Acc_f: 0.8790, Acc_r: 0.9585
[0.8580], [0.9760], 0.8940, 0.9400, [0.8640], 0.9570, [0.7910], 0.9830, 0.9850, 0.9610, Acc_f: 0.8722, Acc_r: 0.9533
------------ Retrained model ------------
[0.0000], [0.0000], 0.9810, 0.9590, [0.0000], 0.9360, [0.0000], 0.9800, 0.9910, 0.9680, Acc_f: 0.0000, Acc_r: 0.9692
[0.0000], [0.0000], 0.9820, 0.9610, [0.0000], 0.9340, [0.0000], 0.9720, 0.9900, 0.9740, Acc_f: 0.0000, Acc_r: 0.9688
[0.0000], [0.0000], 0.9800, 0.9650, [0.0000], 0.9330, [0.0000], 0.9850, 0.9940, 0.9590, Acc_f: 0.0000, Acc_r: 0.9693
Original model Acc_f: 87.47 \pm 0.31
Original model Acc_r: 95.55 \pm 0.22
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 96.91 \pm 0.02


In [53]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Proj_mat_lst =[]
train_targets_list = np.array(train_loader.dataset.targets)[train_loader.sampler.indices]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 100, 125, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)

------------ Trail 0 ------------
Threshold:  0.976
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 118/576
Layer 3 : 303/512
Layer 4 : 77/1024
Layer 5 : 86/2048
Layer 6 : 5/2048
----------------------------------------
Threshold:  0.979
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 6/16
Layer 2 : 152/576
Layer 3 : 411/512
Layer 4 : 157/1024
Layer 5 : 179/2048
Layer 6 : 11/2048
----------------------------------------
Threshold:  0.985
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 9/16
Layer 2 : 231/576
Layer 3 : 465/512
Layer 4 : 241/1024
Layer 5 : 261/2048
Layer 6 : 14/2048
----------------------------------------
Threshold:  0.991
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 :

In [54]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

[0.8490], [0.9780], 0.9090, 0.9310, [0.8380], 0.9580, [0.8260], 0.9850, 0.9880, 0.9570, Acc_f: 0.8728, Acc_r: 0.9547
[train] epoch 0, batch 42, loss 1.382326364517212
[0.1160], [0.0200], 0.9560, 0.9760, [0.6420], 0.9570, [0.5340], 0.9850, 0.9890, 0.9630, Acc_f: 0.3280, Acc_r: 0.9710
[train] epoch 1, batch 42, loss 0.6873600482940674
[0.0310], [0.0040], 0.9730, 0.9760, [0.1330], 0.9560, [0.1190], 0.9840, 0.9890, 0.9640, Acc_f: 0.0718, Acc_r: 0.9737
[train] epoch 2, batch 42, loss 0.5258454084396362
[0.0130], [0.0010], 0.9750, 0.9760, [0.0530], 0.9530, [0.0570], 0.9840, 0.9900, 0.9650, Acc_f: 0.0310, Acc_r: 0.9738
[train] epoch 3, batch 42, loss 0.5510820746421814
[0.0090], [0.0000], 0.9770, 0.9760, [0.0220], 0.9530, [0.0400], 0.9840, 0.9900, 0.9650, Acc_f: 0.0178, Acc_r: 0.9742
[train] epoch 4, batch 42, loss 0.4113083481788635
[0.0050], [0.0000], 0.9780, 0.9760, [0.0110], 0.9520, [0.0320], 0.9830, 0.9900, 0.9650, Acc_f: 0.0120, Acc_r: 0.9740
[train] epoch 5, batch 42, loss 0.4051724374

In [11]:
Acc_r = 100*np.array([0.9740, 0.9727, 0.9690])
Acc_f = 100*np.array([0.0093, 0.0008, 0.0003])

print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')



UNSC Acc_f: 0.35 \pm 0.41
UNSC Acc_r: 97.19 \pm 0.21


# Unlearn 5 classes

In [57]:
num_unlearn = 5
arxiv_name = dict[num_unlearn]['arxiv_name']
args.unlearn_class = dict[num_unlearn]['unlean_class']
Acc_r, Acc_f = np.zeros((2,3)), np.zeros((2,3))
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    Acc_f[0][i], Acc_r[0][i] = test_by_class(model, test_loader, i=args.unlearn_class)

print('------------ Retrained model ------------')
for i in range(3):
    model_r = get_model(args)
    try:
        model_r.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/retrain_model_{arxiv_name}_{args.seeds[i]}.pth'))
        Acc_f[1][i], Acc_r[1][i] = test_by_class(model_r, test_loader, i=args.unlearn_class)
    except:
        print('No retrained model')
        break

print(f'Original model Acc_f: {100*Acc_f[0].mean():.2f} \pm {100*Acc_f[0].std():.2f}')
print(f'Original model Acc_r: {100*Acc_r[0].mean():.2f} \pm {100*Acc_r[0].std():.2f}')

print(f'Retrained model Acc_f: {100*Acc_f[1].mean():.2f} \pm {100*Acc_f[1].std():.2f}')
print(f'Retrained model Acc_r: {100*Acc_r[1].mean():.2f} \pm {100*Acc_r[1].std():.2f}')


[0.8490], [0.9780], 0.9090, 0.9310, [0.8380], 0.9580, [0.8260], 0.9850, 0.9880, [0.9570], Acc_f: 0.8896, Acc_r: 0.9542
[0.8460], [0.9820], 0.9160, 0.9370, [0.8760], 0.9640, [0.8120], 0.9880, 0.9850, [0.9610], Acc_f: 0.8954, Acc_r: 0.9580
[0.8580], [0.9760], 0.8940, 0.9400, [0.8640], 0.9570, [0.7910], 0.9830, 0.9850, [0.9610], Acc_f: 0.8900, Acc_r: 0.9518
------------ Retrained model ------------
[0.0000], [0.0000], 0.9900, 0.8950, [0.0000], 0.8220, [0.0000], 0.9820, 0.9720, [0.0000], Acc_f: 0.0000, Acc_r: 0.9322
[0.0000], [0.0000], 0.9630, 0.9560, [0.0000], 0.9350, [0.0000], 0.9840, 0.8570, [0.0000], Acc_f: 0.0000, Acc_r: 0.9390
[0.0000], [0.0000], 0.9800, 0.9580, [0.0000], 0.9480, [0.0000], 0.9990, 0.9960, [0.0000], Acc_f: 0.0000, Acc_r: 0.9762
Original model Acc_f: 89.17 \pm 0.26
Original model Acc_r: 95.47 \pm 0.26
Retrained model Acc_f: 0.00 \pm 0.00
Retrained model Acc_r: 94.91 \pm 1.93


In [58]:
train_loader, val_loader, test_loader = get_dataloader(args)
remain_train_loader, unlearn_train_loader = split_2_remain_unlearn(args, train_loader)
remain_val_loader, unlearn_val_loader = split_2_remain_unlearn(args, val_loader)
remain_test_loader, unlearn_test_loader = split_2_remain_unlearn(args, test_loader)
remain_class = np.setdiff1d(np.arange(args.num_classes), args.unlearn_class)

Proj_mat_lst =[]
train_targets_list = np.array(train_loader.dataset.targets)[train_loader.sampler.indices]
for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    print(f'------------ Trail {i} ------------')
    merged_feat_mat = []
    for cls_id in range(10): 
        cls_indices = np.where(np.isin(train_targets_list, cls_id))[0]
        cls_indices = train_loader.sampler.indices[cls_indices]
        cls_sampler = torch.utils.data.SubsetRandomSampler(cls_indices)
        cls_loader_dict = torch.utils.data.DataLoader(train_loader.dataset, 
                                                                batch_size=args.batch_size, 
                                                                sampler=cls_sampler)
        if cls_id in args.unlearn_class:
            continue
        for batch, (x, y) in enumerate(cls_loader_dict ):
            x = x.cuda()
            y = y.cuda()
            mat_list = get_representation_matrix(model, 
                                                x, 
                                                batch_list=[24, 100, 100, 100, 125, 125, 125, 250, 250, 256, 256])
            break
        threshold = 0.97 + 0.003*cls_id
        merged_feat_mat = update_GPM(mat_list, threshold, merged_feat_mat)
        proj_mat = [torch.Tensor(np.dot(layer_basis, layer_basis.transpose())) for layer_basis in merged_feat_mat]
        Proj_mat_lst.append(proj_mat)

------------ Trail 0 ------------
Threshold:  0.976


----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 4/16
Layer 2 : 125/576
Layer 3 : 318/512
Layer 4 : 78/1024
Layer 5 : 83/2048
Layer 6 : 5/2048
----------------------------------------
Threshold:  0.979
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 5/16
Layer 2 : 146/576
Layer 3 : 412/512
Layer 4 : 158/1024
Layer 5 : 176/2048
Layer 6 : 11/2048
----------------------------------------
Threshold:  0.985
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 8/16
Layer 2 : 227/576
Layer 3 : 464/512
Layer 4 : 244/1024
Layer 5 : 261/2048
Layer 6 : 14/2048
----------------------------------------
Threshold:  0.991
Skip Updating GPM for layer: 1
----------------------------------------
Gradient Constraints Summary
----------------------------------------
Layer 1 : 8/16
Layer 2 : 257/5

In [59]:
def get_pseudo_label(args, model, x):
    masked_output = model(x)
    masked_output[:, args.unlearn_class] = -np.inf
    pseudo_labels = torch.topk(masked_output, k=1, dim=1).indices
    return pseudo_labels.reshape(-1)

for i in range(3):
    model = get_model(args)
    model.load_state_dict(torch.load(f'./save/{args.dataset}/{args.model_name}/original_model_{arxiv_name_o}_{args.seeds[i]}.pth'))
    sgd_mr_model = deepcopy(model)
    test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)
    optimizer = torch.optim.SGD(sgd_mr_model.parameters(), lr=0.005)
    sgd_mr_model.train()
    for m in sgd_mr_model.modules():
        if isinstance(m, nn.BatchNorm2d) or isinstance(m, nn.BatchNorm1d):
            m.eval()

    model.eval()
    for ep in range(15):
        for batch, (x, y) in enumerate(unlearn_train_loader):
            x = x.cuda()
            y = get_pseudo_label(args, model, x)
            pred_y = sgd_mr_model(x)
            loss = criterion(pred_y, y)
            optimizer.zero_grad()
            loss.backward()
            kk = 0 
            for k, (m,params) in enumerate(sgd_mr_model.named_parameters()):
                if len(params.size())!=1:
                    sz =  params.grad.data.size(0)
                    params.grad.data = params.grad.data - torch.mm(params.grad.data.view(sz,-1),\
                                            Proj_mat_lst[i][kk].cuda()).view(params.size())
                    kk +=1
                elif len(params.size())==1:
                    params.grad.data.fill_(0)
            optimizer.step()
        print('[train] epoch {}, batch {}, loss {}'.format(ep, batch, loss))
        test_by_class(sgd_mr_model, test_loader, i=args.unlearn_class)

[0.8490], [0.9780], 0.9090, 0.9310, [0.8380], 0.9580, [0.8260], 0.9850, 0.9880, [0.9570], Acc_f: 0.8896, Acc_r: 0.9542
[train] epoch 0, batch 52, loss 1.0092248916625977
[0.1350], [0.0240], 0.9600, 0.9760, [0.6360], 0.9630, [0.5310], 1.0000, 0.9890, [0.1110], Acc_f: 0.2874, Acc_r: 0.9776
[train] epoch 1, batch 52, loss 0.5859055519104004
[0.0310], [0.0060], 0.9730, 0.9760, [0.1400], 0.9650, [0.0910], 0.9990, 0.9890, [0.0080], Acc_f: 0.0552, Acc_r: 0.9804
[train] epoch 2, batch 52, loss 0.4705379009246826
[0.0160], [0.0040], 0.9720, 0.9770, [0.0580], 0.9650, [0.0430], 0.9990, 0.9890, [0.0010], Acc_f: 0.0244, Acc_r: 0.9804
[train] epoch 3, batch 52, loss 0.42829957604408264
[0.0110], [0.0020], 0.9750, 0.9770, [0.0310], 0.9640, [0.0280], 0.9990, 0.9890, [0.0000], Acc_f: 0.0144, Acc_r: 0.9808
[train] epoch 4, batch 52, loss 0.38154730200767517
[0.0060], [0.0020], 0.9750, 0.9770, [0.0140], 0.9630, [0.0190], 0.9990, 0.9890, [0.0000], Acc_f: 0.0082, Acc_r: 0.9806
[train] epoch 5, batch 52, lo

In [15]:
Acc_r = 100*np.array([0.9806, 0.9788, 0.9770])
Acc_f = 100*np.array([0.0006, 0.0006, 0.0001])
print(f'UNSC Acc_f: {Acc_f.mean():.2f} \pm {Acc_f.std():.2f}')
print(f'UNSC Acc_r: {Acc_r.mean():.2f} \pm {Acc_r.std():.2f}')

UNSC Acc_f: 0.04 \pm 0.02
UNSC Acc_r: 97.88 \pm 0.15
